<table style="margin: auto; background-color: white;">
    <tr>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1S3xpVbkzpBAG51PyuRyIioecvZiHGSap' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1S3xpVbkzpBAG51PyuRyIioecvZiHGSap' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1S3xpVbkzpBAG51PyuRyIioecvZiHGSap' alt="drawing" width="200" />
      </td>
    </tr>
</table>

#**Causal Graph Fuzzy LLM**


<td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1igs_5aIACphsNn2oZiE10-nRFvnSXv-K' alt="drawing" width="2000" />
</td>




In [1]:
!pip install --quiet git+https://github.com/PatriciaLucas/AutoML.git

!pip install --quiet git+https://petroniocandido/clshq_tk.git

!git clone https://github.com/PatriciaLucas/Fuzzy-Causal-LLM.git

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'Fuzzy-Causal-LLM' already exists and is not an empty directory.


In [2]:
from AUTODCETS import util, feature_selection, datasets, save_database as sd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import clshq_tk
from clshq_tk.modules.fuzzy import GridPartitioner, trimf, gaussmf, training_loop
from clshq_tk.data.regression import RegressionTS
from clshq_tk.common import DEVICE, DEFAULT_PATH, resume, checkpoint, order_window

import sys
sys.path.append('Fuzzy-Causal-LLM')
import fuzzy_causal_text as fcllm

##**Datasets**:

**1.   CLIMATIC_1:** SONDA --- TARGET: glo_avg --- 35.000 SAMPLES --- 12 VARIABLES

**2.   ENERGY_1**: WIND ENERGY PRODUCTION --- TARGET: Power --- 43.800 SAMPLES --- 9 VARIABLES

**3.   IOT_1:** HOUSEHOLD ELECTRICITY CONSUMPTION IN MEXICO --- TARGET: active_power --- 100.000 SAMPLES --- 14 VARIABLES

**4.   ECONOMICS_1:** BITCOIN --- TARGET: AVG --- 2.970 SAMPLES --- 06 VARIABLES



In [17]:
name_dataset = 'ECONOMICS_1'
target = 'AVG'
df = fcllm.upload_dataset(name_dataset)

if name_dataset == 'CLIMATIC_1':
  df = df.head(35000)
elif name_dataset == 'ENERGY_1':
  df = df.head(43800)
elif name_dataset == 'IOT_1':
  df = df.head(100000)
else:
  df = df.head(2970)

max_lags = 20
partitions = 30
path_model = 'model'
database_path = 'database_text.db'
epochs = 20

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
name_model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(name_model)

windows = fcllm.rolling_window(df, 10)

# sd.execute("CREATE TABLE IF NOT EXISTS results(name_dataset TEXT, window INT, predict FLOAT, \
#                real FLOAT)", database_path)

The experiment was conducted to evaluate the methods across $10$ windows of size $0.3 * |D|$ with a $30\%$ overlap throughout the multivariate time series of sample size $|D|$. The windows were divided into training and test sets.

In [18]:
def causal_text(df, name_dataset, target, max_lags, tokenizer):

    variables = df.columns.tolist()
    dict_variables = dict.fromkeys(variables)

    # Causal graph generation
    graph = feature_selection.causal_graph(df.head(2000), target=target, max_lags=max_lags)[target]
    X, y_hat = util.organize_dataset(df, graph, max_lags, target)
    y = df[target].squeeze().tolist()[max_lags:]
    y = np.asarray(y)

    scaler = StandardScaler()
    labels_scaled = scaler.fit_transform(y.reshape(-1, 1))

    inputs = fcllm.create_sequences_input(X, y, tokenizer)

    # Tokenization
    tokenizer.pad_token = tokenizer.eos_token
    input_tokens = tokenizer(
    inputs,
    padding_side='left',
    padding=True,
    truncation=True,
    max_length=1024,
    return_tensors="pt"
    )

    #input_tokens = tokenizer(inputs, padding_side = 'left', padding=True, return_tensors="pt")

    return fcllm.custom_Dataset(input_tokens.input_ids, input_tokens.attention_mask, labels_scaled), scaler, tokenizer, inputs

def text(df, name_dataset, target, max_lags, tokenizer):

    # Complete graph generation
    graph = feature_selection.complete_graph(df.head(2000), target=target, max_lags=max_lags)[target]
    X, y_hat = util.organize_dataset(df, graph, max_lags, target)
    y = df[target].squeeze().tolist()[max_lags:]
    y = np.asarray(y)

    scaler = StandardScaler()
    labels_scaled = scaler.fit_transform(y.reshape(-1, 1))

    inputs = fcllm.create_sequences_input(X, y, tokenizer)

    # Tokenization
    tokenizer.pad_token = tokenizer.eos_token
    #input_tokens = tokenizer(inputs, padding_side = 'left', padding=True, return_tensors="pt")
    input_tokens = tokenizer(
    inputs,
    padding_side='left',
    padding=True,
    truncation=True,
    max_length=1024,
    return_tensors="pt"
    )

    return fcllm.custom_Dataset(input_tokens.input_ids, input_tokens.attention_mask, labels_scaled), scaler, tokenizer, inputs


def fuzzy_causal(df, name_dataset, target, max_lags, tokenizer, partitions):

    variables = df.columns.tolist()
    dict_variables = dict.fromkeys(variables)

    # Fuzzification of time series
    data_fuzzy = pd.DataFrame(columns=variables)
    for v in variables:
        dict_variables[v] = fcllm.fuzzification(pd.DataFrame(df[v]), name_dataset, v, partitions, None)
        data_fuzzy[v] = dict_variables[v][0]

    # Causal graph generation
    graph = feature_selection.causal_graph(df.head(2000), target=target, max_lags=max_lags)[target]
    X, y_hat = util.organize_dataset(data_fuzzy, graph, max_lags, target)
    y = dict_variables[target][1].squeeze().tolist()[max_lags:]
    y = np.asarray(y)

    scaler = StandardScaler()
    labels_scaled = scaler.fit_transform(y.reshape(-1, 1))

    inputs = fcllm.create_sequences_input(X, y, tokenizer)

    # Tokenization
    tokenizer.pad_token = tokenizer.eos_token
    input_tokens = tokenizer(
    inputs,
    padding_side='left',
    padding=True,
    truncation=True,
    max_length=1024,
    return_tensors="pt"
    )

    #input_tokens = tokenizer(inputs, padding_side = 'left', padding=True, return_tensors="pt")

    return fcllm.custom_Dataset(input_tokens.input_ids, input_tokens.attention_mask, labels_scaled), scaler, tokenizer, inputs

In [19]:
# Loop through the windows
for i, window in enumerate(windows):
      if i < 0:  # Enter the number of the last window saved in the database here
        print("window already executed")

      else:

        # Exclude constant series.
        for variable in window.columns:
            if window[variable].max() == window[variable].min():
                window = window.drop(variable, axis=1)
                print(f"Variables {variable} were deleted because they are constant.")

        # Fuzzy-Causal
        ds_F, scaler, tokenizer_F, seq_F = fuzzy_causal(window, name_dataset, target, max_lags, tokenizer, partitions)

        # Text
        ds_T, scaler, tokenizer_T, seq_T = text(window, name_dataset, target, max_lags, tokenizer)

        # Text-Causal
        ds_G, scaler, tokenizer_G, seq_G = causal_text(window, name_dataset, target, max_lags, tokenizer)

        # train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

        break
        # freeze = True # Freeze GPT-2 weights
        # model = fcllm.train_model(train_dataset, name_model, epochs, scaler, freeze, path_model = None)

        # forecasts, real = fcllm.predict(test_dataset, model, tokenizer, target, scaler)

        # for j in range(len(forecasts)):
        #     sd.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?)", (name_dataset, i, forecasts[j], real[j]), database_path)

        # print(f'Save window: {i}')

In [ ]:
print(type(ds_F))
print(dir(ds_F))  # List available methods/attributes
print(ds_F.__getitem__(0))  # Try accessing the first item manually

<class 'fuzzy_causal_text.custom_Dataset'>
['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', 'attention_mask', 'input_ids', 'labels']
{'input_ids': tensor([50256, 50256, 50256, 50256, 50256,    58, 26125,    15, 13872,    15,
         3334,    15, 13872,    15,    60,   657,    13, 41322, 26704, 50256]), 'attention_mask': tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([-1.1405])}


##name_dataset = 'CLIMATIC_1', target = 'glo_avg'

In [6]:
# Assuming tokenizer_F, tokenizer_G, tokenizer_T are the same (GPT-2 tokenizer)
# and ds_F, ds_G, ds_T, seq_F, seq_G, seq_T are available from your code

# Function to calculate sizes
def calculate_sizes(ds, seq, tokenizer, method_name):
    # Token size (total and average, with and without padding)
    total_tokens = sum(len(ds[i]['input_ids']) for i in range(len(ds)))
    total_non_padding_tokens = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in range(len(ds))
    )
    avg_tokens = total_tokens / len(ds)
    avg_non_padding_tokens = total_non_padding_tokens / len(ds)

    # Text size (total and average character length)
    total_chars = sum(len(text) for text in seq)
    avg_chars = total_chars / len(seq)

    print(f"\n{method_name}:")
    print(f"Dataset length (number of samples): {len(ds)}")
    print(f"Total tokens (including padding): {total_tokens}")
    print(f"Total non-padding tokens: {total_non_padding_tokens}")
    print(f"Average tokens per sequence (including padding): {avg_tokens:.2f}")
    print(f"Average non-padding tokens per sequence: {avg_non_padding_tokens:.2f}")
    print(f"Total text size (characters): {total_chars}")
    print(f"Average text size per sequence (characters): {avg_chars:.2f}")

# Calculate for each method
calculate_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_sizes(ds_G, seq_G, tokenizer_G, "GF-LLM (causal_text)")
calculate_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")



CGF-LLM (fuzzy_causal):
Dataset length (number of samples): 10479
Total tokens (including padding): 461076
Total non-padding tokens: 443109
Average tokens per sequence (including padding): 44.00
Average non-padding tokens per sequence: 42.29
Total text size (characters): 786258
Average text size per sequence (characters): 75.03

GF-LLM (causal_text):
Dataset length (number of samples): 10480
Total tokens (including padding): 576400
Total non-padding tokens: 298478
Average tokens per sequence (including padding): 55.00
Average non-padding tokens per sequence: 28.48
Total text size (characters): 660437
Average text size per sequence (characters): 63.02

LLM (text):
Dataset length (number of samples): 10480
Total tokens (including padding): 10731520
Total non-padding tokens: 10730875
Average tokens per sequence (including padding): 1024.00
Average non-padding tokens per sequence: 1023.94
Total text size (characters): 29934389
Average text size per sequence (characters): 2856.33


In [7]:
def calculate_train_test_sizes(ds, seq, tokenizer, method_name):
    # Split the dataset into train (80%) and test (20%)
    train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

    # Token counts (including and excluding padding)
    train_token_count = sum(len(ds[i]['input_ids']) for i in train_dataset.indices)
    train_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in train_dataset.indices
    )
    test_token_count = sum(len(ds[i]['input_ids']) for i in test_dataset.indices)
    test_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in test_dataset.indices
    )

    # Text size (character length)
    train_texts, test_texts = torch.utils.data.random_split(seq, [0.80, 0.20])
    train_char_count = sum(len(text) for text in train_texts)
    test_char_count = sum(len(text) for text in test_texts)

    # Print results
    print(f"\nTrain/Test Split for {method_name}:")
    print(f"Train tokens (including padding): {train_token_count}")
    print(f"Train non-padding tokens: {train_non_padding_token_count}")
    print(f"Test tokens (including padding): {test_token_count}")
    print(f"Test non-padding tokens: {test_non_padding_token_count}")
    print(f"Train text size (characters): {train_char_count}")
    print(f"Test text size (characters): {test_char_count}")

# Calculate for each method
calculate_train_test_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_train_test_sizes(ds_G, seq_G, tokenizer_G, "GF-LLM (causal_text)")
calculate_train_test_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")


Train/Test Split for CGF-LLM (fuzzy_causal):
Train tokens (including padding): 368896
Train non-padding tokens: 354492
Test tokens (including padding): 92180
Test non-padding tokens: 88617
Train text size (characters): 629000
Test text size (characters): 157258

Train/Test Split for GF-LLM (causal_text):
Train tokens (including padding): 461120
Train non-padding tokens: 238072
Test tokens (including padding): 115280
Test non-padding tokens: 60406
Train text size (characters): 528553
Test text size (characters): 131884

Train/Test Split for LLM (text):
Train tokens (including padding): 8585216
Train non-padding tokens: 8584808
Test tokens (including padding): 2146304
Test non-padding tokens: 2146067
Train text size (characters): 23949302
Test text size (characters): 5985087


## name_dataset = 'Ebergy', target = 'power'

In [6]:
# Assuming tokenizer_F, tokenizer_G, tokenizer_T are the same (GPT-2 tokenizer)
# and ds_F, ds_G, ds_T, seq_F, seq_G, seq_T are available from your code

# Function to calculate sizes
def calculate_sizes(ds, seq, tokenizer, method_name):
    # Token size (total and average, with and without padding)
    total_tokens = sum(len(ds[i]['input_ids']) for i in range(len(ds)))
    total_non_padding_tokens = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in range(len(ds))
    )
    avg_tokens = total_tokens / len(ds)
    avg_non_padding_tokens = total_non_padding_tokens / len(ds)

    # Text size (total and average character length)
    total_chars = sum(len(text) for text in seq)
    avg_chars = total_chars / len(seq)

    print(f"\n{method_name}:")
    print(f"Dataset length (number of samples): {len(ds)}")
    print(f"Total tokens (including padding): {total_tokens}")
    print(f"Total non-padding tokens: {total_non_padding_tokens}")
    print(f"Average tokens per sequence (including padding): {avg_tokens:.2f}")
    print(f"Average non-padding tokens per sequence: {avg_non_padding_tokens:.2f}")
    print(f"Total text size (characters): {total_chars}")
    print(f"Average text size per sequence (characters): {avg_chars:.2f}")

# Calculate for each method
calculate_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_sizes(ds_G, seq_G, tokenizer_G, "CG-LLM (causal_text)")
calculate_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")



CGF-LLM (fuzzy_causal):
Dataset length (number of samples): 13119
Total tokens (including padding): 380451
Total non-padding tokens: 353012
Average tokens per sequence (including padding): 29.00
Average non-padding tokens per sequence: 26.91
Total text size (characters): 1276952
Average text size per sequence (characters): 97.34

CG-LLM (causal_text):
Dataset length (number of samples): 13120
Total tokens (including padding): 839680
Total non-padding tokens: 604276
Average tokens per sequence (including padding): 64.00
Average non-padding tokens per sequence: 46.06
Total text size (characters): 1190523
Average text size per sequence (characters): 90.74

LLM (text):
Dataset length (number of samples): 13120
Total tokens (including padding): 13434880
Total non-padding tokens: 13432451
Average tokens per sequence (including padding): 1024.00
Average non-padding tokens per sequence: 1023.81
Total text size (characters): 23194823
Average text size per sequence (characters): 1767.90


In [7]:
def calculate_train_test_sizes(ds, seq, tokenizer, method_name):
    # Split the dataset into train (80%) and test (20%)
    train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

    # Token counts (including and excluding padding)
    train_token_count = sum(len(ds[i]['input_ids']) for i in train_dataset.indices)
    train_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in train_dataset.indices
    )
    test_token_count = sum(len(ds[i]['input_ids']) for i in test_dataset.indices)
    test_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in test_dataset.indices
    )

    # Text size (character length)
    train_texts, test_texts = torch.utils.data.random_split(seq, [0.80, 0.20])
    train_char_count = sum(len(text) for text in train_texts)
    test_char_count = sum(len(text) for text in test_texts)

    # Print results
    print(f"\nTrain/Test Split for {method_name}:")
    print(f"Train tokens (including padding): {train_token_count}")
    print(f"Train non-padding tokens: {train_non_padding_token_count}")
    print(f"Test tokens (including padding): {test_token_count}")
    print(f"Test non-padding tokens: {test_non_padding_token_count}")
    print(f"Train text size (characters): {train_char_count}")
    print(f"Test text size (characters): {test_char_count}")

# Calculate for each method
calculate_train_test_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_train_test_sizes(ds_G, seq_G, tokenizer_G, "CG-LLM (causal_text)")
calculate_train_test_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")


Train/Test Split for CGF-LLM (fuzzy_causal):
Train tokens (including padding): 304384
Train non-padding tokens: 282428
Test tokens (including padding): 76067
Test non-padding tokens: 70584
Train text size (characters): 1021690
Test text size (characters): 255262

Train/Test Split for CG-LLM (causal_text):
Train tokens (including padding): 671744
Train non-padding tokens: 483461
Test tokens (including padding): 167936
Test non-padding tokens: 120815
Train text size (characters): 952364
Test text size (characters): 238159

Train/Test Split for LLM (text):
Train tokens (including padding): 10747904
Train non-padding tokens: 10745699
Test tokens (including padding): 2686976
Test non-padding tokens: 2686752
Train text size (characters): 18564729
Test text size (characters): 4630094


## IOT data

In [12]:
# Assuming tokenizer_F, tokenizer_G, tokenizer_T are the same (GPT-2 tokenizer)
# and ds_F, ds_G, ds_T, seq_F, seq_G, seq_T are available from your code

# Function to calculate sizes
def calculate_sizes(ds, seq, tokenizer, method_name):
    # Token size (total and average, with and without padding)
    total_tokens = sum(len(ds[i]['input_ids']) for i in range(len(ds)))
    total_non_padding_tokens = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in range(len(ds))
    )
    avg_tokens = total_tokens / len(ds)
    avg_non_padding_tokens = total_non_padding_tokens / len(ds)

    # Text size (total and average character length)
    total_chars = sum(len(text) for text in seq)
    avg_chars = total_chars / len(seq)

    print(f"\n{method_name}:")
    print(f"Dataset length (number of samples): {len(ds)}")
    print(f"Total tokens (including padding): {total_tokens}")
    print(f"Total non-padding tokens: {total_non_padding_tokens}")
    print(f"Average tokens per sequence (including padding): {avg_tokens:.2f}")
    print(f"Average non-padding tokens per sequence: {avg_non_padding_tokens:.2f}")
    print(f"Total text size (characters): {total_chars}")
    print(f"Average text size per sequence (characters): {avg_chars:.2f}")

# Calculate for each method
calculate_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_sizes(ds_G, seq_G, tokenizer_G, "CG-LLM (causal_text)")
calculate_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")



CGF-LLM (fuzzy_causal):
Dataset length (number of samples): 29979
Total tokens (including padding): 839412
Total non-padding tokens: 781784
Average tokens per sequence (including padding): 28.00
Average non-padding tokens per sequence: 26.08
Total text size (characters): 3147800
Average text size per sequence (characters): 105.00

CG-LLM (causal_text):
Dataset length (number of samples): 29980
Total tokens (including padding): 1738840
Total non-padding tokens: 1012191
Average tokens per sequence (including padding): 58.00
Average non-padding tokens per sequence: 33.76
Total text size (characters): 2064031
Average text size per sequence (characters): 68.85

LLM (text):
Dataset length (number of samples): 29980
Total tokens (including padding): 30699520
Total non-padding tokens: 30699520
Average tokens per sequence (including padding): 1024.00
Average non-padding tokens per sequence: 1024.00
Total text size (characters): 94793482
Average text size per sequence (characters): 3161.89


In [13]:
def calculate_train_test_sizes(ds, seq, tokenizer, method_name):
    # Split the dataset into train (80%) and test (20%)
    train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

    # Token counts (including and excluding padding)
    train_token_count = sum(len(ds[i]['input_ids']) for i in train_dataset.indices)
    train_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in train_dataset.indices
    )
    test_token_count = sum(len(ds[i]['input_ids']) for i in test_dataset.indices)
    test_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in test_dataset.indices
    )

    # Text size (character length)
    train_texts, test_texts = torch.utils.data.random_split(seq, [0.80, 0.20])
    train_char_count = sum(len(text) for text in train_texts)
    test_char_count = sum(len(text) for text in test_texts)

    # Print results
    print(f"\nTrain/Test Split for {method_name}:")
    print(f"Train tokens (including padding): {train_token_count}")
    print(f"Train non-padding tokens: {train_non_padding_token_count}")
    print(f"Test tokens (including padding): {test_token_count}")
    print(f"Test non-padding tokens: {test_non_padding_token_count}")
    print(f"Train text size (characters): {train_char_count}")
    print(f"Test text size (characters): {test_char_count}")

# Calculate for each method
calculate_train_test_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_train_test_sizes(ds_G, seq_G, tokenizer_G, "CG-LLM (causal_text)")
calculate_train_test_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")


Train/Test Split for CGF-LLM (fuzzy_causal):
Train tokens (including padding): 671552
Train non-padding tokens: 625443
Test tokens (including padding): 167860
Test non-padding tokens: 156341
Train text size (characters): 2518273
Test text size (characters): 629527

Train/Test Split for CG-LLM (causal_text):
Train tokens (including padding): 1391072
Train non-padding tokens: 810007
Test tokens (including padding): 347768
Test non-padding tokens: 202184
Train text size (characters): 1651208
Test text size (characters): 412823

Train/Test Split for LLM (text):
Train tokens (including padding): 24559616
Train non-padding tokens: 24559616
Test tokens (including padding): 6139904
Test non-padding tokens: 6139904
Train text size (characters): 75825454
Test text size (characters): 18968028


##Economics

In [20]:
# Assuming tokenizer_F, tokenizer_G, tokenizer_T are the same (GPT-2 tokenizer)
# and ds_F, ds_G, ds_T, seq_F, seq_G, seq_T are available from your code

# Function to calculate sizes
def calculate_sizes(ds, seq, tokenizer, method_name):
    # Token size (total and average, with and without padding)
    total_tokens = sum(len(ds[i]['input_ids']) for i in range(len(ds)))
    total_non_padding_tokens = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in range(len(ds))
    )
    avg_tokens = total_tokens / len(ds)
    avg_non_padding_tokens = total_non_padding_tokens / len(ds)

    # Text size (total and average character length)
    total_chars = sum(len(text) for text in seq)
    avg_chars = total_chars / len(seq)

    print(f"\n{method_name}:")
    print(f"Dataset length (number of samples): {len(ds)}")
    print(f"Total tokens (including padding): {total_tokens}")
    print(f"Total non-padding tokens: {total_non_padding_tokens}")
    print(f"Average tokens per sequence (including padding): {avg_tokens:.2f}")
    print(f"Average non-padding tokens per sequence: {avg_non_padding_tokens:.2f}")
    print(f"Total text size (characters): {total_chars}")
    print(f"Average text size per sequence (characters): {avg_chars:.2f}")

# Calculate for each method
calculate_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_sizes(ds_G, seq_G, tokenizer_G, "CG-LLM (causal_text)")
calculate_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")



CGF-LLM (fuzzy_causal):
Dataset length (number of samples): 867
Total tokens (including padding): 17340
Total non-padding tokens: 12388
Average tokens per sequence (including padding): 20.00
Average non-padding tokens per sequence: 14.29
Total text size (characters): 43875
Average text size per sequence (characters): 50.61

CG-LLM (causal_text):
Dataset length (number of samples): 868
Total tokens (including padding): 32984
Total non-padding tokens: 17768
Average tokens per sequence (including padding): 38.00
Average non-padding tokens per sequence: 20.47
Total text size (characters): 40643
Average text size per sequence (characters): 46.82

LLM (text):
Dataset length (number of samples): 868
Total tokens (including padding): 846300
Total non-padding tokens: 762734
Average tokens per sequence (including padding): 975.00
Average non-padding tokens per sequence: 878.73
Total text size (characters): 1335281
Average text size per sequence (characters): 1538.34


In [21]:
def calculate_train_test_sizes(ds, seq, tokenizer, method_name):
    # Split the dataset into train (80%) and test (20%)
    train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

    # Token counts (including and excluding padding)
    train_token_count = sum(len(ds[i]['input_ids']) for i in train_dataset.indices)
    train_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in train_dataset.indices
    )
    test_token_count = sum(len(ds[i]['input_ids']) for i in test_dataset.indices)
    test_non_padding_token_count = sum(
        len([t for t in ds[i]['input_ids'] if t != tokenizer.pad_token_id])
        for i in test_dataset.indices
    )

    # Text size (character length)
    train_texts, test_texts = torch.utils.data.random_split(seq, [0.80, 0.20])
    train_char_count = sum(len(text) for text in train_texts)
    test_char_count = sum(len(text) for text in test_texts)

    # Print results
    print(f"\nTrain/Test Split for {method_name}:")
    print(f"Train tokens (including padding): {train_token_count}")
    print(f"Train non-padding tokens: {train_non_padding_token_count}")
    print(f"Test tokens (including padding): {test_token_count}")
    print(f"Test non-padding tokens: {test_non_padding_token_count}")
    print(f"Train text size (characters): {train_char_count}")
    print(f"Test text size (characters): {test_char_count}")

# Calculate for each method
calculate_train_test_sizes(ds_F, seq_F, tokenizer_F, "CGF-LLM (fuzzy_causal)")
calculate_train_test_sizes(ds_G, seq_G, tokenizer_G, "CG-LLM (causal_text)")
calculate_train_test_sizes(ds_T, seq_T, tokenizer_T, "LLM (text)")


Train/Test Split for CGF-LLM (fuzzy_causal):
Train tokens (including padding): 13880
Train non-padding tokens: 9950
Test tokens (including padding): 3460
Test non-padding tokens: 2438
Train text size (characters): 35079
Test text size (characters): 8796

Train/Test Split for CG-LLM (causal_text):
Train tokens (including padding): 26410
Train non-padding tokens: 14256
Test tokens (including padding): 6574
Test non-padding tokens: 3512
Train text size (characters): 32621
Test text size (characters): 8022

Train/Test Split for LLM (text):
Train tokens (including padding): 677625
Train non-padding tokens: 610872
Test tokens (including padding): 168675
Test non-padding tokens: 151862
Train text size (characters): 1068135
Test text size (characters): 267146
